In [6]:
pip install langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install unstructured


  Using cached unstructured-0.16.0-py3-none-any.whl.metadata (24 kB)
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     -------------------------------------- 981.5/981.5 kB 7.7 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------------------------------ --------- 1.3/1.7 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 5.5 MB/s eta 0:00:00
Using cached backoff-2.2.1-py

In [1]:
pip install langchain python-dotenv unstructured azure-openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement azure-openai (from versions: none)
ERROR: No matching distribution found for azure-openai


In [1]:
import os
from typing import List, Dict
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SequentialChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.callbacks import get_openai_callback
from langchain_openai import AzureChatOpenAI

In [2]:
class PDDAnalyzer:
    def __init__(self):
        # Initialize Azure OpenAI
        self.llm = AzureChatOpenAI(
            openai_api_version=os.getenv("AZURE_OPENAI_API_VERSION"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            azure_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
            openai_api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            model_name=os.getenv("LLM_MODEL"),
            temperature=0
        )
        
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=2000,
            chunk_overlap=200
        )
        
    def load_document(self, file_path: str) -> str:
        """Load and process document from specified path"""
        try:
            loader = UnstructuredFileLoader(file_path)
            documents = loader.load()
            splits = self.text_splitter.split_documents(documents)
            return "\n".join([doc.page_content for doc in splits])
        except Exception as e:
            print(f"Error loading document: {str(e)}")
            raise

    def setup_prompts(self) -> Dict[str, PromptTemplate]:
        """Setup all prompt templates"""
        prompts = {
            "summarize": PromptTemplate(
                input_variables=["content"],
                template="""Summarize the following process design document content:
                {content}
                
                Provide a clear and concise summary focusing on:
                1. Main objectives
                2. Key processes
                3. System interactions
                """
            ),
            
            "extract_steps": PromptTemplate(
                input_variables=["content", "summary"],
                template="""Based on this process design document and its summary:
                Summary: {summary}
                Content: {content}
                
                List all the implementation steps in a clear, numbered format.
                For each step, include:
                - Step description
                - Input requirements
                - Expected output
                """
            ),
            
            "technical_solution": PromptTemplate(
                input_variables=["steps", "technology"],
                template="""Given these implementation steps:
                {steps}
                
                Provide a detailed technical solution using {technology} technology.
                Include:
                1. Specific components/modules needed
                2. Implementation approach for each step
                3. Potential technical challenges
                4. Integration points
                """
            ),
            
            "time_estimation": PromptTemplate(
                input_variables=["solution", "skill_level"],
                template="""Based on this technical solution:
                {solution}
                
                Estimate the implementation time for each component considering a {skill_level} developer.
                Provide:
                1. Time estimate per component
                2. Total project duration
                3. Potential bottlenecks
                4. Risk factors affecting timeline
                """
            ),
            
            "cost_estimation": PromptTemplate(
                input_variables=["time_estimate", "technology"],
                template="""Based on the implementation timeline:
                {time_estimate}
                
                Provide a detailed cost estimation for implementing this solution with {technology}.
                Include:
                1. Development costs (using standard rates)
                2. Infrastructure/hosting costs
                3. Third-party services/licenses
                4. Maintenance costs
                5. Total project cost range
                """
            )
        }
        return prompts

    def create_chains(self, prompts: Dict[str, PromptTemplate]) -> SequentialChain:
        """Create sequential chain from prompts"""
        chains = {
            "summary": LLMChain(
                llm=self.llm,
                prompt=prompts["summarize"],
                output_key="summary"
            ),
            "steps": LLMChain(
                llm=self.llm,
                prompt=prompts["extract_steps"],
                output_key="steps"
            ),
            "solution": LLMChain(
                llm=self.llm,
                prompt=prompts["technical_solution"],
                output_key="solution"
            ),
            "time": LLMChain(
                llm=self.llm,
                prompt=prompts["time_estimation"],
                output_key="time_estimate"
            ),
            "cost": LLMChain(
                llm=self.llm,
                prompt=prompts["cost_estimation"],
                output_key="cost_estimate"
            )
        }
        
        return SequentialChain(
            chains=[chains["summary"], chains["steps"], chains["solution"], 
                   chains["time"], chains["cost"]],
            input_variables=["content", "technology", "skill_level"],
            output_variables=["summary", "steps", "solution", "time_estimate", "cost_estimate"]
        )

    def analyze_pdd(self, file_path: str, technology: str, skill_level: str) -> Dict:
        """Main method to analyze PDD document"""
        try:
            content = self.load_document(file_path)
            prompts = self.setup_prompts()
            chain = self.create_chains(prompts)
            
            with get_openai_callback() as cb:
                result = chain({"content": content, 
                              "technology": technology,
                              "skill_level": skill_level})
                print(f"\nTotal Tokens Used: {cb.total_tokens}")
                print(f"Total Cost (USD): ${cb.total_cost}")
            
            return result
        except Exception as e:
            print(f"Error during analysis: {str(e)}")
            raise



In [3]:
# Example usage in Jupyter notebook

# First set up your environment variables
import os

os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] = "62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "DskumarDeployment"
os.environ['OPENAI_TYPE'] = "Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"



In [4]:
# Initialize analyzer
analyzer = PDDAnalyzer()

In [11]:
pip install langchain-unstructured

  Attempting uninstall: unstructured-client
    Found existing installation: unstructured-client 0.26.1
    Uninstalling unstructured-client-0.26.1:
      Successfully uninstalled unstructured-client-0.26.1
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_unstructured import UnstructuredLoader

In [13]:
pip install pillow pi-heif urllib3

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 9.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_unstructured import UnstructuredLoader

In [6]:
# Analyze document
result = analyzer.analyze_pdd(
    file_path=r"C:\Users\817840\OneDrive - Cognizant\Documents\GitHub\ML-AI\Codes\Projects&POC\PGE - Self service chatbot\MRBR followups _Sweden_ PDD_V1.pdf",
    technology="Blue Brism",
    skill_level="intermediate"
)

C:\Users\817840\AppData\Local\Temp\ipykernel_13960\1492333887.py:21: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  loader = UnstructuredFileLoader(file_path)
c:\Users\817840\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Matplotlib is building the font cache; this may take a moment.
C:\Users\817840\AppData\Local\Temp\ipykernel_13960\1492333887.py:106: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and w


Total Tokens Used: 19539
Total Cost (USD): $0.06045299999999999


In [ ]:


# Access results
print("Summary:", result["summary"])
print("\nSteps:", result["steps"])
print("\nTechnical Solution:", result["solution"])
print("\nTime Estimation:", result["time_estimate"])
print("\nCost Estimation:", result["cost_estimate"])

In [12]:
pip install python-magic-bin

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [17]:
pip install unstructured


Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install azure-openai


Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement azure-openai (from versions: none)
ERROR: No matching distribution found for azure-openai


In [19]:
pip install python-magic-bin  # for Windows


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: '#': Expected package name at the start of dependency specifier
    #
    ^


In [20]:
pip install pdf2image


Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install pytesseract


Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install pypdf


Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install docx2txt

Note: you may need to restart the kernel to use updated packages.


In [10]:
pip install "unstructured[all-docs]"


In [ ]:
pip install "unstructured-inference"


In [ ]:
pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6#egg=detectron2


In [ ]:
pip install layoutparser[layoutmodels,tesseract]

In [ ]:
pip install pypdf


In [ ]:
pip install python-docx


In [ ]:
pip install docx2txt